# Clean the Grambow-Green activation dataset 

data from: https://zenodo.org/record/3715478#.XuAZxC3MxTY

paper describing the data: https://doi.org/10.1038/s41597-020-0460-4
paper using the data: J. Phys. Chem. Lett. 2020, 11, 2992−2997

In [1]:
import json
import multiprocessing
from collections import defaultdict
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem
from rxnrep.core.molecule import MoleculeError
from rxnrep.core.reaction import ReactionError, smiles_to_reaction
from rxnrep.data.grapher import (
    get_atom_distance_to_reaction_center,
    get_bond_distance_to_reaction_center,
)
from rxnrep.utils import to_path

RDLogger.logger().setLevel(RDLogger.CRITICAL)  # supress rdkit warnings

Using backend: pytorch


In [2]:
def plot_smiles_reaction(reaction, filename):
    """
    Plot a smiles reaction to file.
    """
    rxn = AllChem.ReactionFromSmarts(reaction, useSmiles=True)
    image = Chem.Draw.ReactionToImage(rxn)
    image.save(filename)

In [3]:
def split_train_val_test(data: List, val_ratio=0.1, test_ratio=0.1, random_seed=35):
    """
    Randomly split dataset into training, validation, and test test.
    """
    assert val_ratio + test_ratio < 1.0, "validation + test >= 1"
    size = len(data)
    num_val = int(size * val_ratio)
    num_test = int(size * test_ratio)
    num_train = size - num_val - num_test

    if random_seed is not None:
        np.random.seed(random_seed)

    idx = np.random.permutation(size)
    train_idx = idx[:num_train]
    val_idx = idx[num_train : num_train + num_val]
    test_idx = idx[num_train + num_val :]

    train_set = [data[i] for i in train_idx]
    val_set = [data[i] for i in val_idx]
    test_set = [data[i] for i in test_idx]

    return train_set, val_set, test_set

In [4]:
def check_convert_to_mp_core_reaction(smi_rxn):
    """
    Check whether a smile reaction can be converted to core reaction.
    """

    try:
        rxn = smiles_to_reaction(
            smi_rxn,
            id=smi_rxn,
            ignore_reagents=True,
            remove_H=False,
            sanity_check=False,
        )

        rxn.check_charge()
        rxn.check_composition()
        rxn.check_atom_map_number()

    except (MoleculeError, ReactionError) as e:
        return None, "Fail converting to core reaction: " + str(e)

    # check we can get labels
    try:
        get_atom_distance_to_reaction_center(rxn)
        get_bond_distance_to_reaction_center(rxn)
    except AssertionError as e:
        return None, "Fail converting to core reaction: " + str(e)

    return {"smi_rxn": smi_rxn, "core_rxn": rxn}, None

In [5]:
def runner(reactions, reaction_ids, func, nprocs=1):
    """
    Run with multiprocess. 
    """
    if nprocs == 1:
        processed_rxns = [func(rxn) for rxn in reactions]
    else:
        with multiprocessing.Pool(nprocs) as p:
            processed_rxns = p.map(func, reactions)

    succeeded = []
    succeeded_ids = []
    failed = []
    failed_ids = []
    for i, (value, error) in enumerate(processed_rxns):
        iid = reaction_ids[i]

        if error is not None:
            failed.append(error)
            failed_ids.append(iid)
        else:
            succeeded.append(value)
            succeeded_ids.append(iid)

    return succeeded, succeeded_ids, failed, failed_ids

### read smiles reaction

In [6]:
def read_file(filename, include_reverse=False):
    """
    Read smiles reactions, activation energy and reaction enthalpy. 
    """

    # read smiles reactions
    df = pd.read_csv(filename, header=0, index_col=0)

    reactants = df["rsmi"].tolist()
    products = df["psmi"].tolist()
    activation_energy = df["ea"].tolist()
    reaction_enthalpy = df["dh"].tolist()

    smiles_rxns = [f"{r}>>{p}" for r, p in zip(reactants, products)]
    
    if include_reverse:
        smiles_rxns += [f"{p}>>{r}" for r, p in zip(reactants, products)]
        activation_energy += [a-r for a,r in zip(activation_energy, reaction_enthalpy)]
        reaction_enthalpy += [-x for x in reaction_enthalpy]

    return smiles_rxns, activation_energy, reaction_enthalpy

In [7]:
# filename = "/Users/mjwen/Documents/Dataset/activation_energy_Green/b97d3.csv"
filename = "/Users/mjwen/Documents/Dataset/activation_energy_Green/wb97xd3.csv"
# filename = "/Users/mjwen/Documents/Dataset/activation_energy_Green/wb97xd3_n200.csv"

path = to_path(filename)
smiles_rxns, activation_energy, reaction_enthalpy = read_file(filename, include_reverse=True)

### check reactions by convering to core Reaction

In [8]:
if __name__ == "__main__":
    multiprocessing.set_start_method("fork", force=True)

    reaction_ids = list(range(len(smiles_rxns)))
    succeeded1, succeeded1_ids, failed1, failed1_ids = runner(
        smiles_rxns, reaction_ids, check_convert_to_mp_core_reaction, nprocs=4,
    )

print("number succeeded:", len(succeeded1))
print("number failed:", len(failed1))

number succeeded: 23906
number failed: 16


In [9]:
failed = failed1
failed_ids = failed1_ids

succeeded = []
for rxn, i in zip(succeeded1, succeeded1_ids):
    succeeded.append({"smi_rxn": rxn["smi_rxn"], "core_rxn": rxn["core_rxn"], "idx": i})

### write succeeded (with original smi input)

In [10]:
fname = path.parent.joinpath(path.stem + "_succeeded" + path.suffix)
with open(fname, "w") as f:
    f.write("index\toriginal_reaction\tcanonical_reaction\n")
    for x in succeeded:
        i = x["idx"]
        f.write(f"{i}\t{smiles_rxns[i]}\t{x['smi_rxn']}\n")

#         # save iamge
#         fname = fname = path.parent.joinpath('image', f'{i}_original' + '.png')
#         plot_smiles_reaction(smiles_rxns[i], fname)
#         fname = fname = path.parent.joinpath('image', f'{i}_edited' + '.png')
#         plot_smiles_reaction(smi, fname)

### write failed (with original smi input)

In [11]:
fname = path.parent.joinpath(path.stem + "_failed" + path.suffix)
with open(fname, "w") as f:
    f.write("index\toriginal_reaction\terror\n")
    for i, error in zip(failed_ids, failed):
        f.write(f"{i}\t{smiles_rxns[i]}\t{error}\n")

### write dataset for training 

In [12]:
def write_dataset_for_training(data: List, activation_energy: List, reaction_enthalpy:List, fname):
    """Write dataset to tsv."""
    smiles = []
    raw_id = []
    activation = []
    enthalpy = []
    for x in data:
            i= x["idx"]
            raw_id.append(i)
            smiles.append(x["smi_rxn"])
            activation.append(activation_energy[i])
            enthalpy.append(reaction_enthalpy[i])

    df = pd.DataFrame({'reaction':smiles, 'activation energy':activation, 'reaction enthalpy':enthalpy, 'raw id':raw_id})
    df.to_csv(fname, index=False, sep='\t')

In [13]:
def get_species(dataset):
    """Get the species in train/test/val dataset."""
    species = set()
    for x in dataset:
        rxn = x["core_rxn"]
        species.update(rxn.species)

    return sorted(species)


def remove_rare_rxn(dataset, species: List[str]):
    """
    Remove rxn in dataset whose species are not in `species`. Typically `species` 
    is from the training set and `dataset` are test/val set. If some species are 
    not in the species of the training set, there are not so many of them and we 
    remove such reactions.
    """
    species = set(species)
    new_dataset = []
    for x in dataset:
        rxn = x["core_rxn"]
        if set(rxn.species).issubset(species):
            new_dataset.append(x)

    return new_dataset

In [14]:
# generate dataset for training

train_set, val_set, test_set = split_train_val_test(succeeded)

tr_fname = path.parent.joinpath(path.stem + "_processed_train.tsv")
write_dataset_for_training(train_set, activation_energy, reaction_enthalpy, tr_fname)

val_fname = path.parent.joinpath(path.stem + "_processed_val.tsv")
write_dataset_for_training(val_set,activation_energy, reaction_enthalpy, val_fname)

te_fname = path.parent.joinpath(path.stem + "_processed_test.tsv")
write_dataset_for_training(test_set,activation_energy, reaction_enthalpy, te_fname)